In [23]:
# =============================================================================
# CELL 1: IMPORTS WITH WINDOWS/VPN FIXES 
# =============================================================================

import sys
import warnings

# Suppress noisy warnings
warnings.filterwarnings('ignore')

# === WINDOWS ASYNC FIX ===
if sys.platform == 'win32':
    import asyncio
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        print("✅ Windows async policy set")
    except Exception as e:
        print(f"⚠️ Async policy warning: {e}")

# === SSL/VPN FIXES ===
import os
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === MAIN IMPORTS ===
import pandas as pd
import numpy as np
import requests
import io
import json
import time

# === YAHOO IMPORTS (with error handling) ===
try:
    from yahooquery import Ticker
    print("✅ yahooquery loaded")
except Exception as e:
    print(f"❌ yahooquery failed: {e}")
    Ticker = None

try:
    import yfinance as yf
    print("✅ yfinance loaded")
except Exception as e:
    print(f"❌ yfinance failed: {e}")
    yf = None

print("\n🎉 All imports complete!")

✅ Windows async policy set
✅ yahooquery loaded
✅ yfinance loaded

🎉 All imports complete!


In [24]:
# =============================================================================
# CELL 2: CONFIGURATION AND SETUP
# =============================================================================


import os
import json



# Saves all configuration settings and helper functions
DATA_FOLDER = "YfinanceDataDump"

# Check if the folder already exists.
if not os.path.exists(DATA_FOLDER):
    try:
        # os.makedirs() creates the folder
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        # If something goes wrong (like no permission), save to current folder
        print(f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}")
        DATA_FOLDER = "."  # The dot means "current folder"

# 2. FILE PATHS
# -------------
# These are the names of the files where we'll save our results
# os.path.join() combines the folder name with the file name properly
# (handles / vs \ on different operating systems automatically)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache.json")    # Stores data we've already fetched
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks.csv")   # Best quality stocks
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks.csv")       # Good quality stocks
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks.csv")         # Lower quality stocks
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks.csv")  # Analyst favorites
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks.csv") # Value picks
DEEPVAL_CSV = os.path.join(DATA_FOLDER, "Deep_Value_Gems.csv")    # Intersection of Buffett + Burry
# 3. UNIVERSE FILTERS - Minimum requirements for a stock to be considered

MIN_PRICE = 2.00              # Stock price must be at least $2 (avoid "penny stocks")
MIN_VOLUME = 1_000_000          # At least 1,000,000 shares traded per day (on average)

MIN_CAP = 300_000_000          # Market cap at least $300 million (company value)
                              # Market Cap = Stock Price × Number of Shares
MIN_CURRENT_RATIO = 1.2       # Current Ratio must be > 1.2
                              # Current Ratio = Current Assets / Current Liabilities
                              # If > 1, the company can pay its short-term bills
MAX_PE_RATIO = 100.0          # P/E ratio must be less than 100
                              # P/E = Price / Earnings Per Share
                              

# 4. SAFETY THRESHOLDS - Additional quality filters
# -------------------------------------------------
MIN_INTEREST_COVERAGE = 1.5   # Interest Coverage > 1.5 means company earns 1.5x what it owes in interest
                              # Interest Coverage = EBIT / Interest Expense
                              # If < 1, company can't pay its interest bills = bad!
MIN_ROIC = 0.05               # ROIC (Return on Invested Capital) > 5%
                              # ROIC = Operating Profit / Capital Invested
                              # Shows how well management uses money to generate returns
FORTRESS_MARGIN_THRESHOLD = 0.05  # Operating Margin > 5% for "Fortress" (best) tier
                                   # Operating Margin = Operating Income / Revenue
                                   # Shows what % of sales becomes actual profit

# Sectors we want to EXCLUDE (skip)
# Financial Services and Real Estate have very different financial structures
EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']

CACHE_EXPIRY_DAYS = 30  # Re-fetch data if our saved data is older than 30 days


# ==========================================
# HELPER FUNCTIONS - Reusable code blocks
# ==========================================


def load_cache():
    """
    Load previously saved financial data from our cache file.
    Returns:
        dict: A dictionary of saved data, or empty dict {} if no cache exists
    """
    # Check if the cache file exists
    if os.path.exists(CACHE_FILE):
        try:
            # 'r' means "read mode" - we're reading the file, not writing to it
            with open(CACHE_FILE, 'r') as f:
                # json.load() reads the JSON file and converts it to a Python dictionary
                return json.load(f)
        except:
            # If something goes wrong reading the file, return empty dictionary
            return {}
    # If file doesn't exist, return empty dictionary
    return {}

def save_cache(cache_data):
    """
    Save our financial data to a file so we can reuse it later.
    
    Args:
        cache_data (dict): The data we want to save
    """
    try:
        # 'w' means "write mode" - creates the file or overwrites existing
        with open(CACHE_FILE, 'w') as f:
            # json.dump() converts the Python dictionary to JSON format and saves it
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculate the Altman Z-Score - a formula that predicts bankruptcy risk.
    
    Created by Professor Edward Altman in 1968.
    - Z > 2.99: "Safe Zone" - company is financially healthy
    - Z between 1.81 and 2.99: "Grey Zone" - uncertain
    - Z < 1.81: "Distress Zone" - high risk of bankruptcy
    
    The Formula: Z = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    
    Where:
    A = Working Capital / Total Assets (liquidity)
    B = Retained Earnings / Total Assets (profitability over time)
    C = EBIT / Total Assets (operating efficiency)
    D = Market Value of Equity / Total Liabilities (market confidence)
    E = Sales / Total Assets (asset efficiency)
    
    Args:
        bs: Balance Sheet data (DataFrame from yfinance)
        fin: Financial data (DataFrame from yfinance)  
        market_cap: Market capitalization in dollars
        
    Returns:
        float: The Z-Score, or 0 if calculation fails
    """
    try:
        # Helper function to safely get values from DataFrames
        # Sometimes the data has different names, so we try multiple
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    # .iloc[0] gets the first value (most recent year)
                    return df.loc[k].iloc[0]
            return 0

        # Get values from Balance Sheet (bs) and Financials (fin)
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])  # Earnings Before Interest & Taxes
        total_revenue = get_val(fin, ['Total Revenue'])
        
        # Can't divide by zero, so return 0 if missing key data
        if total_assets == 0 or total_liab == 0: 
            return 0

        # Calculate each component of the Z-Score formula
        # A: Working Capital (can company pay short-term bills?)
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings (accumulated profits over the years)
        B = retained_earnings / total_assets
        
        # C: EBIT (operating profitability)
        C = ebit / total_assets
        
        # D: Market Cap vs Debt (how much does market trust vs owe?)
        D = market_cap / total_liab
        
        # E: Revenue (is the company using its assets efficiently?)
        E = total_revenue / total_assets
        
        # The final formula with Altman's coefficients
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    
    except Exception as e:
        return 0  # Return 0 if any calculation fails
    

    
# =============================================================================
def apply_trend_alignment(df_input):
    """
    Calculates the 200-day SMA and filters for stocks trading ABOVE it.
    
    Args:
        df_input: DataFrame containing a 'Ticker' column.
        
    Returns:
        DataFrame: Filtered list of stocks in an Uptrend.
    """
    if df_input is None or df_input.empty:
        print("   [Trend Alignment] Input DataFrame is empty. Skipping.")
        return df_input

    tickers = df_input['Ticker'].tolist()
    print(f"\n--- TREND ALIGNMENT: Checking 200-Day SMA for {len(tickers)} stocks ---")
    
    # 1. Fetch History (Need at least 200 trading days, so 1y is perfect)
    #    auto_adjust=True fixes prices for splits/dividends
    try:
        data = yf.download(tickers, period="1y", interval="1d", progress=False, group_by='ticker', auto_adjust=True)
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return df_input

    trend_data = []

    for ticker in tickers:
        try:
            # Handle MultiIndex if multiple tickers, or single Index if one ticker
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need at least 200 days of data
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history (<200 days).")
                continue

            # 2. Calculate 200-Day SMA
            # .rolling(window=200).mean() averages the last 200 closing prices
            sma_200 = df_hist['Close'].rolling(window=200).mean().iloc[-1]
            current_price = df_hist['Close'].iloc[-1]

            # 3. The Filter Condition: Price > SMA 200
            # We want stocks that have "reclaimed" their trend
            is_uptrend = current_price > sma_200
            
            # Calculate how far above/below the SMA it is (as a %)
            distance_pct = round(((current_price - sma_200) / sma_200) * 100, 2)

            if is_uptrend:
                # Get the original row data
                base_row = df_input[df_input['Ticker'] == ticker].iloc[0].to_dict()
                
                # Add technical metrics
                base_row['SMA_200'] = round(sma_200, 2)
                base_row['Trend_Dist_%'] = distance_pct
                
                trend_data.append(base_row)

        except Exception as e:
            continue
            
    # Create the new filtered DataFrame
    df_uptrend = pd.DataFrame(trend_data)
    
    if not df_uptrend.empty:
        print(f"   [Result] {len(df_uptrend)}/{len(tickers)} stocks are in a Long-Term Uptrend.")
        # Sort by strength of trend (distance above SMA)
        df_uptrend = df_uptrend.sort_values(by='Trend_Dist_%', ascending=True)
    else:
        print("   [Result] No stocks passed the Trend Alignment filter.")
        
    return df_uptrend

In [25]:
# =============================================================================
# CELL 3: STEP 1 - FETCH US STOCK UNIVERSE
# =============================================================================


def get_combined_universe():
    """
    Get a list of all US stock tickers from NASDAQ.
    
    The NASDAQ provides a file with all traded stocks.
    US tickers don't need a suffix (like .TO for Toronto).
    
    Returns:
        list: List of US stock symbols like ['AAPL', 'MSFT', 'GOOGL', ...]
    """
    print("--- STEP 1: Fetching North American Universe ---")
    tickers = []
    
    # Fetch from NASDAQ's official list
    try:
        url_us = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        
        # Download the file
        s = requests.get(url_us).content
        
        # Read it as a CSV with pipe (|) as the separator
        # The file looks like: Symbol|Security Name|ETF|Test Issue|...
        df_us = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        
        # Filter out test issues and ETFs (we only want real stocks)
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        
        # Get symbols, clean them, and keep only short ones (< 5 characters)
        # Long symbols are usually special securities we don't want
        us_list = [x.replace('$', '-') for x in df_us['Symbol'].astype(str) if len(x) < 5]
        
        tickers.extend(us_list)  # Add to our list
        print(f"   -> Found {len(us_list)} US stocks.")
        
    except:
        print("   -> Error fetching USA list.")

    return tickers

In [26]:
# =============================================================================
# CELL 4: STEP 2 - LIGHTWEIGHT FILTER (QUICK SCREENING)
# =============================================================================
# This is the first filter - it quickly eliminates stocks that don't meet
# basic requirements. It's "lightweight" because it uses fast bulk data fetching.

def get_initial_survivors(tickers):
    """
    Filter stocks using basic criteria. 
    Args:
        tickers (list): List of all stock symbols to check
        
    Returns:
        DataFrame: Table of stocks that passed all filters with their metrics
    """
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    
    chunk_size = 500  # Process stocks in batches of 500 at a time
    survivors = []    # This list will hold stocks that pass all filters
    
    # Split our list into chunks (batches) divides a large request into smaller parts
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    # Process each chunk
    for i, chunk in enumerate(chunks):
        # Print progress every 5 batches
        if i % 5 == 0: 
            print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        
        try:
            # Create a Ticker object for all stocks in this chunk
            # asynchronous=True means it fetches data for multiple stocks simultaneously
            yq = Ticker(chunk, asynchronous=True)
            
            # Get multiple types of data at once (this is the "bulk fetch")
            # These are different Yahoo Finance data modules
            df_modules = yq.get_modules(
                'summaryProfile summaryDetail financialData price defaultKeyStatistics'
            )
            
            # Loop through each stock's data
            for symbol, data in df_modules.items():
                # If data is just a string, it means there was an error
                if isinstance(data, str): 
                    continue
                
                try:
                    # Extract the stock price
                    # The .get() method returns the value if it exists, or 0 if it doesn't
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: 
                        price = 0
                    
                    # Extract average daily trading volume
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                        # Try alternative location for volume data
                        vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    # Extract market capitalization (total company value)
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: 
                        cap = 0
                    
                    # Extract sector (Technology, Healthcare, etc.)
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    
                    # Get financial data
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)      # Current Ratio
                    op_margins = fin_data.get('operatingMargins', 0)  # Operating Margin (as decimal)
                    if curr_ratio is None: 
                        curr_ratio = 0
                    if op_margins is None: 
                        op_margins = 0

                    # Get P/E ratio
                    pe = data.get('summaryDetail', {}).get('trailingPE')

                    # ====== APPLY FILTERS ======
                    # Each 'continue' statement skips to the next stock (odd yes but continue skipping to the next stock essentially means it failed the filter and we move on)
                    
                    # Skip if P/E is too high (overvalued)
                    if pe is not None and pe > MAX_PE_RATIO: 
                        continue
                    
                    # Skip if price is too low (penny stock)
                    if price < MIN_PRICE: 
                        continue
                    
                    # Skip if company is too small
                    if cap < MIN_CAP: 
                        continue
                    
                    # Skip if not enough trading volume (hard to buy/sell)
                    if vol < MIN_VOLUME: 
                        continue
                    
                    # Skip if in excluded sectors
                    # any() returns True if ANY item in the list is True
                    if any(x in sector for x in EXCLUDED_SECTORS): 
                        continue
                    
                    # Skip if current ratio is too low (can't pay bills)
                    if curr_ratio < MIN_CURRENT_RATIO: 
                        continue
                    
                    # Skip if operating margin is zero or negative (not profitable)
                    if op_margins <= 0: 
                        continue
                    
                    # If we get here, the stock passed ALL filters!
                    # Add it to our survivors list
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),  # Convert to percentage
                        'P/E': round(pe, 2) if pe else 0,
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)  # Convert to billions
                    })
                    
                except:
                    continue  # Skip this stock if any error occurs
                    
        except:
            continue  # Skip this batch if any error occurs
    
    # Convert our list of dictionaries to a pandas DataFrame (table)
    return pd.DataFrame(survivors)

In [27]:
# =============================================================================
# CELL 5: STEP 3 - DEEP FINANCIAL ANALYSIS
# =============================================================================
# This is the detailed analysis phase. For each stock that passed Step 2,
# we download detailed financial statements and calculate advanced metrics.

def get_advanced_metrics(survivor_df):
    """
    Perform deep financial analysis on stocks that passed initial screening.

    
    Args:
        survivor_df: DataFrame of stocks from Step 2
        
    Returns:
        DataFrame: Stocks with full analysis and tier classification
    """
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    
    # Load our cached data (data we've already fetched before)
    cache = load_cache()
    current_time = time.time()  # Current time in seconds since 1970 (Unix timestamp)
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400  # Convert days to seconds (86400 sec/day)
    
    final_data = []  # Will store our results
    
    # Loop through each stock
    for i, ticker in enumerate(tickers):
        # Print progress every 20 stocks
        if i % 20 == 0: 
            print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # Uncomment the line below if you're getting throttled (too many requests)
        # time.sleep(0.75)  # Wait 0.75 seconds between requests
        
        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            """
            Determine tier based on Margins AND Financial Health (Z-Score).
            
            STRICT RULES:
            1. Fortress = High Margin (>5%) AND Safe Z-Score (>2.99)
            2. Strong   = Positive Margin (>0%) AND Acceptable Z-Score (>1.81)
            3. Risky    = Fails either margins or safety
            """
            
            # Extract Z-Score from the metrics dictionary
            z_val = metrics.get('z_score', 0)
            
            # 1. IMMEDIATE FAILURES (Hard Safety Stops)
            # If a company can't pay interest or has low return on capital, it's Risky.
            if metrics['int_cov'] < MIN_INTEREST_COVERAGE: 
                return "Risky"
            if metrics['roic'] < MIN_ROIC: 
                return "Risky"
            
            # 2. FORTRESS CRITERIA (The "Perfect" Stock)
            # Must have BOTH strong margins AND a safe Z-Score
            if is_fortress_margin and z_val >= 2.99:
                return "Fortress"
                
            # 3. STRONG CRITERIA (The "Good" Stock)
            # Must have at least positive margins AND be out of the "Distress Zone"
            # We use 1.81 because Z < 1.81 implies high bankruptcy risk
            elif is_pos_margin and z_val >= 1.81:
                return "Strong"
            
            # 4. FALLTHROUGH
            # If it failed the above, it's Risky (either unprofitable or unsafe balance sheet)
            else:
                return "Risky"
        # Check if we have cached data for this stock that's not expired
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: 
                continue  # Skip if previous fetch failed

        # FETCH NEW DATA using yfinance
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials      # Income statement data
            bs = stock.balance_sheet    # Balance sheet data
            
            # Check if Yahoo actually gave us data
            if fin.empty or bs.empty:
                print(f"   No data for {ticker} (skipping)")
                continue
            
            # --- CALCULATE 4-YEAR AVERAGE OPERATING MARGIN ---
            # We look at multiple years to see if profitability is consistent
            try:
                # Try to get Operating Income (might be called different things)
                if 'Operating Income' in fin.index:
                    op_income_history = fin.loc['Operating Income']
                elif 'EBIT' in fin.index:
                    op_income_history = fin.loc['EBIT']
                else:
                    op_income_history = pd.Series([0])

                # Get Revenue for same periods
                revenue_history = fin.loc['Total Revenue']
                
                # Calculate margin for each year: Operating Income / Revenue
                yearly_margins = (op_income_history / revenue_history).dropna()
                
                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()  # Average of all years
                    
                    # Check if average meets our thresholds
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                is_fortress_margin = False
                is_positive_margin = False

            # --- STANDARD CALCULATIONS ---
            def get_item(df, keys):
                """Helper to get values that might have different names."""
                for k in keys:
                    if k in df.index: 
                        return df.loc[k].iloc[0]
                return 0

            # Get needed values
            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            # Calculate Interest Coverage Ratio
            # This tells us if the company can pay its interest bills
            int_exp = abs(int_exp)  # Make positive (expenses are often negative)
            if int_exp == 0: 
                int_cov = 100  # No debt = infinite coverage, use 100 as placeholder
            else: 
                int_cov = ebit / int_exp
            
            # Calculate ROIC (Return on Invested Capital)
            # Shows how efficiently management uses capital
            invested_cap = total_assets - curr_liab  # Simplified invested capital
            if invested_cap <= 0: 
                roic = 0
            else: 
                roic = ebit / invested_cap
            
            # Calculate Altman Z-Score
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000  # Convert back to dollars
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            # Store metrics in cache for future use
            metrics = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            cache[ticker] = metrics
            
            # Determine final tier based on all metrics
            tier = determine_tier_history(metrics, is_fortress_margin, is_positive_margin)

            # Add all data to our final results
            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Avg Margin (4Y)': round(avg_margin * 100, 2) if 'avg_margin' in locals() else 0,
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue  # Skip if any error

    # Save updated cache to file
    save_cache(cache)
    
    return pd.DataFrame(final_data)

In [28]:
# =============================================================================
# CELL 6: MAIN EXECUTION
# =============================================================================
# This is where we actually RUN all the functions we defined above.
# The 'if __name__ == "__main__":' is a Python convention - it means
# "only run this code if this file is being run directly, not imported"

if __name__ == "__main__":
    # STEP 1: Get list of all US stocks
    tickers = get_combined_universe()
    
    if len(tickers) > 0:
        # STEP 2: Apply quick filters
        survivors_df = get_initial_survivors(tickers)
        
        if not survivors_df.empty:
            print(f"\n Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            # STEP 3: Deep financial analysis
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                # Sort results: First by Tier (alphabetically), then by Z-Score (highest first)
                final_results = final_results.sort_values(
                    by=['Tier', 'Z-Score'], 
                    ascending=[True, False]  # True = A-Z, False = highest first
                )
                
                # Split into three categories based on tier
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                risky_df = final_results[final_results['Tier'] == 'Risky'].copy()
                
                # Save to CSV files (spreadsheet format)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)  # index=False means don't save row numbers
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)
                    
                    print("\n" + "="*60)
                    print("RESULTS GENERATED")
                    print("="*60)
                    print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                    
                except Exception as e:
                    print(f"\n  Error Saving Files: {e}")
                
                # Set display options for pandas (so we can see more data)
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                # Show preview of fortress stocks
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))  # .head(15) shows first 15 rows
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching North American Universe ---


   -> Found 6008 US stocks.

--- STEP 2: Running 'Lightweight' Filter on 6008 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 6/13...
 -> Processing Batch 11/13...

 Step 2 Complete. 555 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 555 Survivors ---
 -> Analyzing 1/555: A...
 -> Analyzing 21/555: ALGM...
 -> Analyzing 41/555: AQN...
 -> Analyzing 61/555: AXL...
 -> Analyzing 81/555: BMY...
 -> Analyzing 101/555: CC...
 -> Analyzing 121/555: CNXC...
 -> Analyzing 141/555: CXM...
 -> Analyzing 161/555: DT...
 -> Analyzing 181/555: ETN...
 -> Analyzing 201/555: FOLD...
 -> Analyzing 221/555: GILD...
 -> Analyzing 241/555: HAL...
 -> Analyzing 261/555: HSIC...
 -> Analyzing 281/555: ISRG...
 -> Analyzing 301/555: LFST...
 -> Analyzing 321/555: MGY...
 -> Analyzing 341/555: NEM...
 -> Analyzing 361/555: OKTA...
 -> Analyzing 381/555: PGNY...
 -> Analyzing 401/555: QRVO...
 -> Analyzing 421/555: RUN...
 -> Analyzing 441/555: SLNO...
 -> Analyzing 461/55

In [29]:
# =============================================================================
# CELL 6a: FORTRESS PULLBACK SCAN (RSI + TREND)
# =============================================================================
# This scan looks for high-quality "Fortress" stocks that are:
# 1. In a long-term Uptrend (Price > 200 SMA)
# 2. In a short-term Pullback (RSI < 40)
# This is the "Buy the Dip" strategy for quality assets.

import yfinance as yf
import pandas as pd
import numpy as np

def apply_fortress_pullback(df_input, rsi_threshold=40):
    """
    Calculates RSI (14-day) and 200-day SMA.
    Filters for stocks where Price > SMA200 AND RSI < Threshold.
    """
    if df_input is None or df_input.empty:
        print("   [Fortress Pullback] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input['Ticker'].tolist()
    print(f"\n--- FORTRESS PULLBACK: Scanning {len(tickers)} stocks (RSI < {rsi_threshold}) ---")
    
    # 1. Fetch History (Need ~1 year for SMA 200)
    try:
        data = yf.download(tickers, period="1y", interval="1d", progress=False, group_by='ticker', auto_adjust=True)
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    pullback_candidates = []

    for ticker in tickers:
        try:
            # Handle Data Structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]: continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need enough data
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history.")
                continue

            # 2. Calculate 200-Day SMA (Trend Filter)
            sma_200 = df_hist['Close'].rolling(window=200).mean().iloc[-1]
            current_price = df_hist['Close'].iloc[-1]
            
            # CONDITION 1: Must be in an Uptrend
            if current_price < sma_200:
                continue

            # 3. Calculate RSI (14-Day)
            # Calculate daily price changes
            delta = df_hist['Close'].diff()
            
            # Separate gains and losses
            gain = (delta.where(delta > 0, 0))
            loss = (-delta.where(delta < 0, 0))
            
            # Calculate Exponential Moving Average (Wilder's Smoothing usually uses alpha=1/14)
            avg_gain = gain.ewm(com=13, adjust=False).mean()
            avg_loss = loss.ewm(com=13, adjust=False).mean()
            
            # Calculate RS and RSI
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))
            
            current_rsi = rsi.iloc[-1]

            # CONDITION 2: Must be Oversold (Pullback)
            if current_rsi < rsi_threshold:
                # Get base data
                base_row = df_input[df_input['Ticker'] == ticker].iloc[0].to_dict()
                
                base_row['RSI_14'] = round(current_rsi, 2)
                base_row['SMA_200'] = round(sma_200, 2)
                # Distance from SMA (Buy Zone Depth)
                base_row['Dist_to_SMA_%'] = round(((current_price - sma_200) / sma_200) * 100, 2)
                
                pullback_candidates.append(base_row)

        except Exception as e:
            continue
            
    return pd.DataFrame(pullback_candidates)

# =========================================
# EXECUTION
# =========================================

# Check if Fortress DataFrame exists
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the scan
    # We use 40 as the threshold because quality stocks rarely drop to 30
    Fortress_Dip_DF = apply_fortress_pullback(fortress_df, rsi_threshold=40)
    
    if not Fortress_Dip_DF.empty:
        # Sort by Lowest RSI (Deepest Dip)
        Fortress_Dip_DF = Fortress_Dip_DF.sort_values(by='RSI_14', ascending=True)
        
        # Save to CSV
        PULLBACK_CSV = "YfinanceDataDump/Fortress_Pullback_Candidates_USA.csv"
        Fortress_Dip_DF.to_csv(PULLBACK_CSV, index=False)
        
        print(f"\nSUCCESS: Pullback candidates saved to '{PULLBACK_CSV}'")
        print(f"Original Fortress Count: {len(fortress_df)} -> Pullback Count: {len(Fortress_Dip_DF)}")
        
        cols = ['Ticker', 'Price', 'RSI_14', 'SMA_200', 'Dist_to_SMA_%', 'Tier']
        print("\n--- FORTRESS STOCKS 'ON SALE' ---")
        print(Fortress_Dip_DF[cols].head(15))
    else:
        print("\nRESULT: 0 stocks passed. No Fortress stocks are currently oversold (RSI < 40).")
else:
    print("Error: 'fortress_df' not found. Please run the Main Execution cell first.")


--- FORTRESS PULLBACK: Scanning 220 stocks (RSI < 40) ---

SUCCESS: Pullback candidates saved to 'YfinanceDataDump/Fortress_Pullback_Candidates_USA.csv'
Original Fortress Count: 220 -> Pullback Count: 10

--- FORTRESS STOCKS 'ON SALE' ---
  Ticker    Price  RSI_14  SMA_200  Dist_to_SMA_%      Tier
7   WLTH    9.500   14.38      NaN            NaN  Fortress
8   VSNT   33.040   22.74      NaN            NaN  Fortress
5   QCOM  159.420   32.30   158.62           0.50  Fortress
0   ISRG  535.068   34.70   514.74           3.94  Fortress
3   NBIX  132.390   35.37   132.19           0.15  Fortress
4    ULS   75.000   35.38    71.79           4.47  Fortress
1    APP  568.760   35.68   482.72          17.82  Fortress
9   TGNA   18.850   36.52    18.35           2.73  Fortress
2     ZM   80.960   36.79    80.14           1.03  Fortress
6   URBN   69.480   38.30    68.67           1.17  Fortress


In [30]:
# =============================================================================
# CELL 7: STEP 4 - ANALYST RATINGS FILTER
# =============================================================================
# This adds another layer of analysis: what do professional analysts think?
# Analyst Rating Scale (typically):
#   1.0 = Strong Buy
#   2.0 = Buy  
#   3.0 = Hold
#   4.0 = Sell
#   5.0 = Strong Sell

def get_analyst_fortress_from_var(df_input):
    """
    Fetch analyst ratings for stocks and filter for those rated "Buy" or better.
    
    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)
        
    Returns:
        DataFrame: Stocks with analyst ratings and upside potential
    """
    working_df = df_input.copy()
    tickers = working_df['Ticker'].tolist()
    
    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks ---")
    print("    (Fetching serially to avoid throttling...)")
    
    analyst_data = []
    
    for i, ticker in enumerate(tickers):
        # Print progress every 10 stocks
        if i % 10 == 0: 
            print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info  # Get all available info for the stock
            
            # Extract analyst recommendation (1.0 to 5.0 scale)
            rec_mean = info.get('recommendationMean')
            target_price = info.get('targetMeanPrice')  # Average price target
            current_price = info.get('currentPrice')
            
            # Filter: Only keep stocks rated 2.5 or better (Buy or Strong Buy)
            if rec_mean is None or rec_mean > 2.5: 
                continue
            
            # Calculate upside potential
            # Upside = (Target - Current) / Current * 100
            upside = 0
            if target_price and current_price:
                upside = round(((target_price - current_price) / current_price) * 100, 2)
            
            # Get the original data and add new columns
            base_row = working_df[working_df['Ticker'] == ticker].iloc[0].to_dict()
            base_row['Analyst_Rating'] = rec_mean
            base_row['Target_Price'] = target_price
            base_row['Upside_%'] = upside
            
            analyst_data.append(base_row)
            time.sleep(0.2)  # Small delay to not be timed out of Yahoo's servers
            
        except Exception:
            continue
            
    return pd.DataFrame(analyst_data)

# ==========================================
# EXECUTE THE ANALYST FILTER
# ==========================================

# Check if fortress_df exists from the previous step
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)
    
    if not Analyst_Fortress_DF.empty:
        # Sort by highest upside potential
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(by='Upside_%', ascending=False)
        
        print("\n  Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.5)")
        
        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print(f"Saved to '{ANALYST_CSV}'")
        
        # Show top picks
        cols = ['Ticker', 'Price', 'Analyst_Rating', 'Upside_%', 'Target_Price', 'Tier']
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("  'fortress_df' not found. Please run Steps 1-3 first.")


--- STEP 4: Fetching Analyst Ratings for 220 Stocks ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/220: WPM...
 -> Analyst Scan 11/220: ANET...
 -> Analyst Scan 21/220: CPRX...
 -> Analyst Scan 31/220: META...
 -> Analyst Scan 41/220: LLY...
 -> Analyst Scan 51/220: TXN...
 -> Analyst Scan 61/220: REGN...
 -> Analyst Scan 71/220: AEM...
 -> Analyst Scan 81/220: ULS...
 -> Analyst Scan 91/220: PAAS...
 -> Analyst Scan 101/220: ABT...
 -> Analyst Scan 111/220: ECL...
 -> Analyst Scan 121/220: LRN...
 -> Analyst Scan 131/220: ATI...
 -> Analyst Scan 141/220: DKS...
 -> Analyst Scan 151/220: GNRC...
 -> Analyst Scan 161/220: TOL...
 -> Analyst Scan 171/220: TMHC...
 -> Analyst Scan 181/220: TPR...
 -> Analyst Scan 191/220: BLDR...
 -> Analyst Scan 201/220: BURL...
 -> Analyst Scan 211/220: BIRK...

  Analyst Scan Complete!
Found 186 stocks with Buy Ratings (Score < 2.5)
Saved to 'YfinanceDataDump\Analyst_Fortress_Picks.csv'
    Ticker    Price  Analyst_Rating  Upsid

In [31]:
# =============================================================================
# CELL 8: BUFFETT VALUE SCAN
# =============================================================================
# This filter looks for stocks Warren Buffett might like:
# - Trading BELOW book value (P/B < 1.0) means you're buying $1 of assets for less than $1
# - Positive Return on Equity (ROE) shows the company is profitable
# - Reasonable debt levels (Debt/Equity < 100%)

def get_buffett_value_picks(df_input):
    """
    Find deep value stocks trading below their book value.
    
    Book Value = Total Assets - Total Liabilities
    P/B Ratio = Stock Price / Book Value per Share
    
    If P/B < 1.0, you're theoretically buying the company for less than
    what it would be worth if you sold all its assets and paid all debts.
    
    Args:
        df_input: DataFrame of stocks to analyze
        
    Returns:
        DataFrame: Deep value stocks sorted by P/B ratio
    """
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Process in chunks for speed
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # Get key statistics and financial data
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core "Value" Rule)
                        pb = stats.get('priceToBook')
                        if pb is None or pb >= 1.0 or pb <= 0: 
                            continue

                        # 2. Positive ROE (Return on Equity - company makes money)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: 
                            continue

                        # 3. Reasonable Debt (not overleveraged)
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: 
                            continue

                        # Get base data and add value metrics
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: 
                        continue
        except: 
            continue

    return pd.DataFrame(buffett_candidates)

# Run the Buffett scan
if 'final_results' in locals() and not final_results.empty:
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks")
        
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
    else:
        print("\n  No stocks passed the Buffett filter.")
else:
    print(" 'final_results' not found. Please run Steps 1-3 first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 551 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 18 Deep Value Stocks

--- DEEP VALUE PICKS ---
   Ticker   Price  P/B Ratio  ROE %  Debt/Eq %                  Sector      Tier
9    ACHC   11.68       0.34   3.69      74.49              Healthcare     Risky
0    GMAB   32.25       0.35  29.41       2.47              Healthcare  Fortress
8      KT   19.49       0.40   5.08      60.17  Communication Services     Risky
5     SSL    6.31       0.44   5.07      76.56         Basic Materials     Risky
7    ANGI   12.49       0.55   3.42      54.03  Communication Services     Risky
4     HLX    7.09       0.66   2.71      39.52                  Energy     Risky
15     WB   10.75       0.67  12.37      47.28  Communication Services    Strong
13     MT   48.73       0.68   4.72      26.23         Basic Materials    Strong
16   MOMO    6.99     

In [32]:
# =============================================================================
# CELL 8a: BUFFETT TREND ALIGNMENT (200-DAY SMA)
# =============================================================================
# This cell takes the 'Deep Value' candidates found in the previous step and
# filters out any that are in a long-term downtrend (Price < 200-Day SMA).
# This helps avoid "Value Traps" - stocks that are cheap because they are dying.

import yfinance as yf
import pandas as pd

def apply_trend_alignment(df_input):
    """
    Calculates the 200-day SMA and filters for stocks trading ABOVE it.
    """
    if df_input is None or df_input.empty:
        print("   [Trend Alignment] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input['Ticker'].tolist()
    print(f"\n--- TREND ALIGNMENT: Checking 200-Day SMA for {len(tickers)} Buffett candidates ---")
    
    # 1. Fetch History (Need at least 200 trading days, so 1y is perfect)
    try:
        # group_by='ticker' ensures the dataframe structure is consistent
        data = yf.download(tickers, period="1y", interval="1d", progress=False, group_by='ticker', auto_adjust=True)
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    trend_data = []

    for ticker in tickers:
        try:
            # Handle data structure: if multiple tickers, it uses MultiIndex
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                # If only one ticker, yfinance returns a flat dataframe
                df_hist = data.copy()

            # Need at least 200 days of data to calculate the SMA
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history (<200 days).")
                continue

            # 2. Calculate 200-Day SMA
            # .rolling(window=200).mean() averages the last 200 closing prices
            sma_200 = df_hist['Close'].rolling(window=200).mean().iloc[-1]
            current_price = df_hist['Close'].iloc[-1]

            # 3. The Filter Condition: Price > SMA 200
            # We want stocks that have "reclaimed" their trend
            is_uptrend = current_price > sma_200
            
            # Calculate how far above/below the SMA it is (as a %)
            distance_pct = round(((current_price - sma_200) / sma_200) * 100, 2)

            if is_uptrend:
                # Get the original row data from the Buffett DataFrame
                base_row = df_input[df_input['Ticker'] == ticker].iloc[0].to_dict()
                
                # Add technical metrics to the row
                base_row['SMA_200'] = round(sma_200, 2)
                base_row['Trend_Dist_%'] = distance_pct
                
                trend_data.append(base_row)

        except Exception as e:
            continue
            
    # Create the new filtered DataFrame
    return pd.DataFrame(trend_data)

# =========================================
# EXECUTION
# =========================================

# Check if the Buffett DataFrame exists from the previous cell
if 'Buffett_Value_DF' in locals() and not Buffett_Value_DF.empty:
    
    # Run the filter
    Buffett_Trend_DF = apply_trend_alignment(Buffett_Value_DF)
    
    if not Buffett_Trend_DF.empty:
        # Sort by trend strength (how far above the 200 SMA they are)
        Buffett_Trend_DF = Buffett_Trend_DF.sort_values(by='Trend_Dist_%', ascending=True)
        
        # Save to a new CSV
        TREND_CSV = "YfinanceDataDump/Buffett_Trend_Aligned_USA.csv"
        Buffett_Trend_DF.to_csv(TREND_CSV, index=False)
        
        print(f"\nSUCCESS: Filtered list saved to '{TREND_CSV}'")
        print(f"Original Count: {len(Buffett_Value_DF)} -> Trend Aligned Count: {len(Buffett_Trend_DF)}")
        
        # Display the survivors
        cols = ['Ticker', 'Price', 'P/B Ratio', 'SMA_200', 'Trend_Dist_%', 'Tier']
        print("\n--- BUFFETT STOCKS IN UPTREND ---")
        print(Buffett_Trend_DF[cols].head(15))
    else:
        print("\nRESULT: 0 stocks passed. All Buffett picks are currently in a downtrend.")
else:
    print("Error: 'Buffett_Value_DF' not found. Please run the Buffett Scan cell first.")


--- TREND ALIGNMENT: Checking 200-Day SMA for 18 Buffett candidates ---

SUCCESS: Filtered list saved to 'YfinanceDataDump/Buffett_Trend_Aligned_USA.csv'
Original Count: 18 -> Trend Aligned Count: 15

--- BUFFETT STOCKS IN UPTREND ---
   Ticker   Price  P/B Ratio  SMA_200  Trend_Dist_%      Tier
1      KT   19.49       0.40    19.26          1.17     Risky
11    SXC    7.91       0.96     7.79          1.59    Strong
14   TMHC   63.27       1.00    61.74          2.47  Fortress
12   TGNA   18.85       0.97    18.35          2.73  Fortress
13    KBH   61.32       0.99    58.20          5.37     Risky
4      WB   10.75       0.67    10.18          5.56    Strong
8     MHK  122.84       0.91   114.52          7.27    Strong
3     HLX    7.09       0.66     6.53          8.50     Risky
9    DINO   48.63       0.95    44.31          9.75     Risky
2     SSL    6.31       0.44     5.51         14.47     Risky
10   SBLK   20.37       0.96    17.74         14.82     Risky
0    GMAB   32.25   

In [62]:
# =============================================================================
# CELL 9: INSIDER TRADING FILTER
# =============================================================================
# Insiders (executives, directors, major shareholders) sometimes have better
# information about their company. When they BUY their own stock, it can be
# a positive sign. When they SELL, it might be neutral (paying taxes) or negative.

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def filter_for_insider_buying(tickers):
    """
    Find stocks where company insiders are NET BUYERS.
    
    Net Buying = Total shares bought - Total shares sold
    If positive, insiders are accumulating shares (bullish signal).
    
    Args:
        tickers: List of stock symbols to check
        
    Returns:
        DataFrame: Stocks with positive insider buying
    """
    print(f" Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []
    
    # Process in smaller chunks to avoid timeout
    chunk_size = 25
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            
            # Fetch insider transaction data
            df_insiders = yq.insider_transactions
            
            # Fetch current prices
            price_data = yq.price
            
            # Skip if no data
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'):
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                # Get transactions for this stock
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                # Separate purchases and sales
                # The text typically says "Purchase" or "Sale"
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.contains(
                    "Purchase", case=False, na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.contains(
                    "Sale", case=False, na=False)]
                
                # Calculate total buy/sell volume
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                # Get current price
                current_price = None
                try:
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get('regularMarketPrice', None)
                except:
                    current_price = None

                # Only keep if net buying is positive
                if buy_vol > sell_vol:
                    insider_picks.append({
                        'Ticker': symbol,
                        'Current_Price': current_price,
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception as e:
            continue

    return pd.DataFrame(insider_picks)

# Run the insider filter
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()

else:
    print("'fortress_df' and 'strong_df' not found.")

Fortress_insiders_1 = filter_for_insider_buying(target_tickers)
print(f"Created 'Fortress_insiders' with {len(Fortress_insiders_1)} rows.")
display(Fortress_insiders_1)

 Scanning 220 stocks for Insider Buying...
Created 'Fortress_insiders' with 12 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,MU,362.750,3,167433.0
1,CDE,22.580,21,1650770.0
2,HXL,82.590,6,14327.0
3,GIL,63.220,105,1426104.0
4,SHLS,9.270,3,8335.0
5,CGAU,16.130,87,4297682.0
6,RAL,53.530,1,2000.0
7,TTD,35.477,10,3557415.0
8,FSM,10.420,10,2116207.0
9,OPCH,36.030,8,18954.0


In [67]:
# =============================================================================
# CELL 9a: INSIDER TRADING FILTER (STRONG LIST ONLY | MIN 2 BUYS)
# =============================================================================
import warnings
import pandas as pd
import time
from yahooquery import Ticker

warnings.simplefilter(action='ignore', category=FutureWarning)

def filter_for_strong_insider_buying(tickers):
    """
    Find stocks where insiders are NET BUYERS with HIGH CONVICTION.
    Criteria:
    1. Net Buying (Buys > Sells)
    2. Conviction (At least 2 separate Buy transactions)
    """
    print(f" Scanning {len(tickers)} stocks for High-Conviction Insider Buying...")
    insider_picks = []
    
    # Conservative chunk size for stability
    chunk_size = 15
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        try:
            # Delay to prevent API blocking
            time.sleep(1.0)
            
            yq = Ticker(chunk, asynchronous=True)
            df_insiders = yq.insider_transactions
            price_data = yq.price
            
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'):
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                # Robust lowercase search
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.lower().str.contains("purchase", na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.lower().str.contains("sale", na=False)]
                
                # === NEW FILTER: MINIMUM 2 BUY TRANSACTIONS ===
                if len(buys) < 2:
                    continue
                
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                # Logic: Net Positive Buying
                if buy_vol > sell_vol:
                    current_price = None
                    try:
                        if isinstance(price_data, dict) and symbol in price_data:
                            current_price = price_data[symbol].get('regularMarketPrice', None)
                    except:
                        current_price = 0

                    insider_picks.append({
                        'Ticker': symbol,
                        'Current_Price': current_price,
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception as e:
            continue

    return pd.DataFrame(insider_picks)

# =========================================
# EXECUTION (STRONG_DF ONLY)
# =========================================

target_tickers = []

# Check for 'strong_df' (or variations of the name)
if 'strong_df' in locals() and not strong_df.empty:
    print(f"Source: 'strong_df' ({len(strong_df)} tickers)")
    target_tickers = strong_df['Ticker'].tolist()

else:
    print("Error: 'strong_df' variable not found. Please run the Strong Filter cell first.")

# Run Scan
if len(target_tickers) > 0:
    # Remove duplicates and Sort
    unique_tickers = sorted(list(set(target_tickers)))
    
    Strong_insiders = filter_for_strong_insider_buying(unique_tickers)
    
    print(f"\nFinal Result: Found {len(Strong_insiders)} Strong stocks with 2+ Insider Buys.")
    
    if not Strong_insiders.empty:
        # Sort by Conviction
        Strong_insiders = Strong_insiders.sort_values(by='Net_Shares_Bought', ascending=False)
        display(Strong_insiders)

Source: 'strong_df' (107 tickers)
 Scanning 107 stocks for High-Conviction Insider Buying...

Final Result: Found 13 Strong stocks with 2+ Insider Buys.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
4,CSTM,22.16,78,41706013.0
11,SLGN,42.86,72,13136746.0
6,EGO,40.28,119,5087258.0
0,AMRZ,54.53,17,1259880.0
1,ATAT,36.89,11,703246.0
12,TEX,60.39,3,147507.0
5,DOW,27.57,4,39639.0
9,LYB,50.91,3,24183.0
8,LEVI,21.76,5,22600.0
10,NWSA,26.25,2,12803.0


In [65]:

Fortress_insiders = pd.concat([Fortress_insiders_1, Strong_insiders], ignore_index=True)
display(Fortress_insiders)

,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,MU,362.750,3,167433.0
1,CDE,22.580,21,1650770.0
2,HXL,82.590,6,14327.0
3,GIL,63.220,105,1426104.0
4,SHLS,9.270,3,8335.0
5,CGAU,16.130,87,4297682.0
6,RAL,53.530,1,2000.0
7,TTD,35.477,10,3557415.0
8,FSM,10.420,10,2116207.0
9,OPCH,36.030,8,18954.0


In [66]:
# =============================================================================
# CELL 9b: RELATIVE VOLUME (RVOL) CATALYST SCAN
# =============================================================================
# This checks your "Insider Picks" for unusual institutional activity.
# RVOL > 1.5 means volume is 150% of normal (institutions are active).
# RVOL > 2.0 is a massive "Ignition" signal.

import yfinance as yf
import pandas as pd

def apply_rvol_catalyst(df_input, rvol_threshold=1.5):
    """
    Calculates Relative Volume (RVOL).
    RVOL = Current Volume / Average Volume (30-day).
    """
    if df_input is None or df_input.empty:
        print("   [RVOL Catalyst] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input['Ticker'].tolist()
    print(f"\n--- RVOL CATALYST: Checking volume for {len(tickers)} Insider picks ---")
    
    # 1. Fetch History (Need ~2 months to get a clean 30-day average)
    try:
        data = yf.download(tickers, period="3mo", interval="1d", progress=False, group_by='ticker', auto_adjust=True)
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    catalyst_data = []

    for ticker in tickers:
        try:
            # Handle data structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]: continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need at least 30 days of volume data
            if len(df_hist) < 30:
                continue

            # 2. Get Volumes
            # Current Volume (Last completed bar)
            current_vol = df_hist['Volume'].iloc[-1]
            
            # Average Volume (Previous 30 days, excluding today to avoid skewing the average)
            avg_vol = df_hist['Volume'].iloc[-31:-1].mean()
            
            # Avoid division by zero
            if avg_vol == 0:
                continue
                
            # 3. Calculate RVOL
            rvol = current_vol / avg_vol
            
            # 4. Filter
            if rvol > rvol_threshold:
                # Get base data
                base_row = df_input[df_input['Ticker'] == ticker].iloc[0].to_dict()
                
                base_row['RVOL'] = round(rvol, 2)
                base_row['Avg_Vol_30D'] = int(avg_vol)
                base_row['Current_Vol'] = int(current_vol)
                
                catalyst_data.append(base_row)

        except Exception as e:
            continue
            
    return pd.DataFrame(catalyst_data)

# =========================================
# EXECUTION
# =========================================

# We check 'Fortress_insiders' first (Best List)
# If that doesn't exist, we check the raw 'Fortress_insiders' list
if 'Fortress_insiders' in locals() and not Fortress_insiders.empty:
    target_df = Fortress_insiders
    print("Source: Analyst-Vetted Insider List")
elif 'Fortress_insiders' in locals() and not Fortress_insiders.empty:
    target_df = Fortress_insiders
    print("Source: Raw Insider List")
else:
    target_df = pd.DataFrame()

if not target_df.empty:
    # Run the scan
    # Threshold 1.5 = 150% of normal volume
    Insider_Catalyst_DF = apply_rvol_catalyst(target_df, rvol_threshold=1.5)
    
    if not Insider_Catalyst_DF.empty:
        # Sort by highest RVOL (Biggest Catalyst)
        Insider_Catalyst_DF = Insider_Catalyst_DF.sort_values(by='RVOL', ascending=False)
        
        # Save results
        CATALYST_CSV = "YfinanceDataDump/Insider_RVOL_Catalysts_USA.csv"
        Insider_Catalyst_DF.to_csv(CATALYST_CSV, index=False)
        
        print(f"\nSUCCESS: Catalyst candidates saved to '{CATALYST_CSV}'")
        print(f"Original Insider Count: {len(target_df)} -> Catalyst Count: {len(Insider_Catalyst_DF)}")
        
        # --- FIX IS HERE: Changed 'Price' to 'Current_Price' ---
        cols = ['Ticker', 'Current_Price', 'RVOL', 'Insider_Buys_Count', 'Net_Shares_Bought']
        
        # Only add Analyst_Verdict if it actually exists in the dataframe
        if 'Analyst_Verdict' in Insider_Catalyst_DF.columns:
            cols.append('Analyst_Verdict')
            
        print("\n--- INSIDER STOCKS WITH HIGH VOLUME SPIKES ---")
        print(Insider_Catalyst_DF[cols].head(15))
    else:
        print("\nRESULT: 0 stocks passed. No Insider picks are currently seeing high relative volume (>1.5x).")
else:
    print("Error: No Insider DataFrame found. Please run the Insider Scan cell first.")

Source: Analyst-Vetted Insider List

--- RVOL CATALYST: Checking volume for 25 Insider picks ---

SUCCESS: Catalyst candidates saved to 'YfinanceDataDump/Insider_RVOL_Catalysts_USA.csv'
Original Insider Count: 25 -> Catalyst Count: 5

--- INSIDER STOCKS WITH HIGH VOLUME SPIKES ---
  Ticker  Current_Price  RVOL  Insider_Buys_Count  Net_Shares_Bought
4   AXTA          33.57  1.97                   7             7366.0
0     MU         362.75  1.66                   3           167433.0
2   SHLS           9.27  1.64                   3             8335.0
3   HSIC          79.98  1.59                   5            16257.0
1    CDE          22.58  1.50                  21          1650770.0


In [34]:

# ==========================================
# 10. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders
        
    tickers = df_insiders['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print(" No Analyst ratings found for these tickers.")
            return df_insiders # Return original if no data found
            
        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"  Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"  Error in Analyst Filter: {e}")
        return df_insiders

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list just in case need to update periodically
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(insider_winners, max_score=2.5)
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(f"\n  Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)")
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

 Scanning 220 stocks for Insider Buying...
  Analyst Filter: 12 -> 10 stocks (Min Rating: Buy).

  Final List: 10 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
9,OPCH,36.030,8,18954.0,1.50000,strong_buy
3,GIL,63.220,105,1426104.0,1.53846,buy
10,CRH,122.970,3,413316.0,1.58333,buy
0,MU,362.750,3,167433.0,1.59524,buy
1,CDE,22.580,21,1650770.0,1.83333,buy
4,SHLS,9.270,3,8335.0,1.95238,buy
7,TTD,35.477,10,3557415.0,2.13158,buy
6,RAL,53.530,1,2000.0,2.25000,buy
8,FSM,10.420,10,2116207.0,2.50000,buy
5,CGAU,16.130,87,4297682.0,2.50000,buy


In [36]:
# =============================================================================
# CELL 11: BURRY EV/EBITDA FILTER For Fortress Stocks
# =============================================================================
# EV/EBITDA is a valuation metric popular with hedge fund manager Michael Burry.
# 
# EV = Enterprise Value = Market Cap + Debt - Cash
#      (What it would cost to buy the whole company)
# 
# EBITDA = Earnings Before Interest, Taxes, Depreciation, and Amortization
#          (Proxy for operating cash flow)
# 
# EV/EBITDA tells you how many years of cash flow it would take to buy the company.
# Lower is better (cheaper stock).
# We compare each stock to its SECTOR AVERAGE to find relative value.

def filter_burry_ev_ebitda(df_input):
    """
    Find stocks trading at a discount to their sector average.
    
    Logic: A stock with EV/EBITDA of 8x is "cheap" in a sector 
           where the average is 15x.
    
    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)
        
    Returns:
        DataFrame: Stocks cheaper than their sector average
    """
    if df_input is None or df_input.empty:
        print(" Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")
    
    tickers = df_input['Ticker'].tolist()
    
    # Fetch data in bulk
    try:
        yq = Ticker(tickers, asynchronous=True)
        data = yq.get_modules('defaultKeyStatistics financialData summaryDetail')
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []
    
    for ticker in tickers:
        try:
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str): 
                continue
            
            stats = ticker_data.get('defaultKeyStatistics', {})
            fin_data = ticker_data.get('financialData', {})
            summary = ticker_data.get('summaryDetail', {})

            # Try to get pre-calculated EV/EBITDA
            ev_ebitda = stats.get('enterpriseToEbitda')
            
            # If not available, calculate manually
            if ev_ebitda is None:
                try:
                    market_cap = summary.get('marketCap')
                    total_debt = fin_data.get('totalDebt')
                    total_cash = fin_data.get('totalCash')
                    ebitda = fin_data.get('ebitda')
                    
                    if all(v is not None for v in [market_cap, total_debt, total_cash, ebitda]):
                        if ebitda != 0:
                            # EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                except:
                    pass

            # Only include positive values (profitable companies)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({
                    'Ticker': ticker,
                    'EV/EBITDA': round(ev_ebitda, 2)
                })
        except:
            continue
            
    df_vals = pd.DataFrame(ev_data)
    
    if df_vals.empty:
        print("Could not retrieve EV/EBITDA data.")
        return pd.DataFrame()

    # Merge with sector data
    merged_df = pd.merge(df_input, df_vals, on='Ticker', how='inner')
    
    # Calculate sector averages
    print("\n--- SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby('Sector')['EV/EBITDA'].mean().reset_index()
    sector_stats.rename(columns={'EV/EBITDA': 'Sector_Avg_EV_EBITDA'}, inplace=True)
    sector_stats['Sector_Avg_EV_EBITDA'] = sector_stats['Sector_Avg_EV_EBITDA'].round(2)
    
    print(sector_stats.to_string(index=False))
    
    # Merge with sector averages
    final_df = pd.merge(merged_df, sector_stats, on='Sector', how='left')
    
    # Filter: Keep only stocks CHEAPER than sector average
    burry_picks = final_df[final_df['EV/EBITDA'] < final_df['Sector_Avg_EV_EBITDA']].copy()
    
    # Calculate discount percentage
    # Discount = 1 - (Stock EV/EBITDA / Sector Average)
    burry_picks['Discount_%'] = round(
        (1 - (burry_picks['EV/EBITDA'] / burry_picks['Sector_Avg_EV_EBITDA'])) * 100, 2
    )
    
    # Sort by biggest discount
    burry_picks = burry_picks.sort_values(by='Discount_%', ascending=False)
    
    return burry_picks

# Run the Burry filter
if 'fortress_df' in locals() and not fortress_df.empty:
    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)
    
    if not Fortress_Burry_EV_EBITDA.empty:
        print(f"\n✅ Found {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks")
        display(Fortress_Burry_EV_EBITDA[['Ticker', 'Sector', 'Price', 'EV/EBITDA', 
                                          'Sector_Avg_EV_EBITDA', 'Discount_%', 'Tier']])

Analyzing EV/EBITDA for 220 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.97
Communication Services                 19.16
     Consumer Cyclical                 14.37
    Consumer Defensive                 15.98
                Energy                  9.27
            Healthcare                 20.85
           Industrials                 20.93
            Technology                 22.16

✅ Found 129 Undervalued Stocks


,Ticker,Sector,Price,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Tier
180,VSNT,Communication Services,33.040,1.83,19.16,90.45,Fortress
48,CALM,Consumer Defensive,77.920,1.65,15.98,89.67,Fortress
168,WLTH,Technology,9.500,2.83,22.16,87.23,Fortress
204,PDD,Consumer Cyclical,106.703,1.98,14.37,86.22,Fortress
31,AVGO,Technology,351.710,4.97,22.16,77.57,Fortress
61,PLAB,Technology,34.530,6.58,22.16,70.31,Fortress
158,FSM,Basic Materials,10.420,4.83,15.97,69.76,Fortress
175,PPC,Consumer Defensive,41.370,5.34,15.98,66.58,Fortress
20,CPRX,Healthcare,22.700,7.09,20.85,66.00,Fortress
210,HAFN,Industrials,5.870,7.91,20.93,62.21,Fortress


In [37]:
# =============================================================================
# CELL 11a: BURRY TECHNICAL SCAN (TREND + SQUEEZE)
# =============================================================================
# This combines two strategies into one efficient pass:
# 1. SAFETY: Filters out downtrends (Price < 200 SMA).
# 2. OPPORTUNITY: Calculates "Volatility Squeeze" to find coiling stocks.

import yfinance as yf
import pandas as pd
import numpy as np

def apply_burry_technicals(df_input):
    """
    Fetches 1y history ONCE.
    Applies Trend Filter (Price > SMA200).
    Calculates Squeeze Metrics (Bandwidth).
    """
    if df_input is None or df_input.empty:
        print("   [Burry Technicals] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input['Ticker'].tolist()
    print(f"\n--- BURRY TECHNICALS: Analyzing {len(tickers)} candidates ---")
    
    # 1. Fetch History (1 Year is enough for both SMA200 and Squeeze)
    try:
        data = yf.download(tickers, period="1y", interval="1d", progress=False, group_by='ticker', auto_adjust=True)
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    results_data = []

    for ticker in tickers:
        try:
            # Handle data structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]: continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need sufficient data
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history (<200 days).")
                continue

            # --- METRIC 1: TREND (SMA 200) ---
            sma_200 = df_hist['Close'].rolling(window=200).mean().iloc[-1]
            current_price = df_hist['Close'].iloc[-1]
            
            # HARD FILTER: Must be in Uptrend
            if current_price < sma_200:
                continue
                
            dist_sma_pct = ((current_price - sma_200) / sma_200) * 100

            # --- METRIC 2: SQUEEZE (Bollinger Bands) ---
            sma_20 = df_hist['Close'].rolling(window=20).mean()
            std_20 = df_hist['Close'].rolling(window=20).std()
            
            upper = sma_20 + (2 * std_20)
            lower = sma_20 - (2 * std_20)
            bandwidth = (upper - lower) / sma_20
            
            # Check Squeeze Status (Lowest 15% of last 6 months)
            recent_bw = bandwidth.tail(126) # ~6 months
            current_bw = recent_bw.iloc[-1]
            squeeze_threshold = recent_bw.quantile(0.15)
            is_squeezing = current_bw <= squeeze_threshold

            # --- COMPILE RESULT ---
            # Get base fundamental data
            base_row = df_input[df_input['Ticker'] == ticker].iloc[0].to_dict()
            
            # Add Technical Data
            base_row['Price'] = round(current_price, 2)
            base_row['SMA_200'] = round(sma_200, 2)
            base_row['Trend_Dist_%'] = round(dist_sma_pct, 2)
            base_row['Bandwidth_%'] = round(current_bw * 100, 2)
            base_row['In_Squeeze'] = "YES" if is_squeezing else "No"
            
            results_data.append(base_row)

        except Exception as e:
            continue
            
    return pd.DataFrame(results_data)

# =========================================
# EXECUTION
# =========================================

if 'Fortress_Burry_EV_EBITDA' in locals() and not Fortress_Burry_EV_EBITDA.empty:
    
    # Run the consolidated scan
    Burry_Technicals_DF = apply_burry_technicals(Fortress_Burry_EV_EBITDA)
    
    if not Burry_Technicals_DF.empty:
        # Sort by "Squeeziness" (Bandwidth), but favor those in Squeeze first
        # We sort by In_Squeeze (descending, so YES comes first) then Bandwidth (ascending)
        Burry_Technicals_DF = Burry_Technicals_DF.sort_values(
            by=['In_Squeeze', 'Bandwidth_%'], 
            ascending=[False, True]
        )
        
        # Save results
        TECH_CSV = "YfinanceDataDump/Burry_Technical_Picks_USA.csv"
        Burry_Technicals_DF.to_csv(TECH_CSV, index=False)
        
        print(f"\nSUCCESS: Technical picks saved to '{TECH_CSV}'")
        print(f"Original Undervalued Count: {len(Fortress_Burry_EV_EBITDA)}")
        print(f"Survivors (Uptrend Only): {len(Burry_Technicals_DF)}")


--- BURRY TECHNICALS: Analyzing 129 candidates ---

SUCCESS: Technical picks saved to 'YfinanceDataDump/Burry_Technical_Picks_USA.csv'
Original Undervalued Count: 129
Survivors (Uptrend Only): 96


In [38]:

# ==========================================
# 12. ANALYST FILTER FUNCTION FOR Burry EV/EBITDA PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if Fortress_Burry_EV_EBITDA.empty:
        return Fortress_Burry_EV_EBITDA
        
    tickers = Fortress_Burry_EV_EBITDA['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return Fortress_Burry_EV_EBITDA # Return original if no data found
            
        # Merge with the Fortress DataFrame
        merged = pd.merge(Fortress_Burry_EV_EBITDA, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return Fortress_Burry_EV_EBITDA

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'Fortress_Burry_EV_EBITDA' in locals() and not Fortress_Burry_EV_EBITDA.empty:
    target_tickers = Fortress_Burry_EV_EBITDA['Ticker'].tolist()


# B. Run Insider Filter
Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not Fortress_Burry_EV_EBITDA.empty:
    Fortress_Burry_Analyst_buy = filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5)
else:
    Fortress_Burry_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_Burry_Analyst_buy.empty:
    print(f"\n🚀 Final List: {len(Fortress_Burry_Analyst_buy)} stocks (Fortress + Burry + Analyst Buy Rating)")
    display(Fortress_Burry_Analyst_buy)
else:
    print("No stocks passed all filters.")

Analyzing EV/EBITDA for 220 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.97
Communication Services                 19.16
     Consumer Cyclical                 14.37
    Consumer Defensive                 15.98
                Energy                  9.27
            Healthcare                 20.85
           Industrials                 20.93
            Technology                 22.16
✅ Analyst Filter: 125 -> 104 stocks (Min Rating: Buy).

🚀 Final List: 104 stocks (Fortress + Burry + Analyst Buy Rating)


,Ticker,Tier,Price,P/E,Sector,Z-Score,ROIC %,Op Margin %,Avg Margin (4Y),Curr Ratio,Int Cov,Mkt Cap (B),EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Analyst_Score,Analyst_Verdict
8,CPRX,Fortress,22.700,13.27,Healthcare,16.04,26.70,44.66,36.56,6.622,100.00,2.79,7.09,20.85,66.00,1.00000,strong_buy
5,PLAB,Fortress,34.530,15.14,Technology,7.86,13.54,24.15,26.03,5.366,4033.44,2.04,6.58,22.16,70.31,1.00000,strong_buy
103,DHT,Fortress,13.260,10.69,Energy,3.63,15.43,37.76,18.35,2.411,6.99,2.13,8.16,9.27,11.97,1.20000,strong_buy
72,Q,Fortress,97.250,37.26,Technology,9.53,7.85,21.71,19.13,1.781,3.21,20.37,16.08,22.16,27.44,1.22222,strong_buy
54,SGHC,Fortress,9.940,23.67,Consumer Cyclical,8.53,31.26,23.52,10.14,1.750,32.04,5.03,9.41,14.37,34.52,1.25000,strong_buy
4,AVGO,Fortress,351.710,73.58,Technology,12.19,17.00,31.77,39.71,1.705,8.08,1667.55,4.97,22.16,77.57,1.26531,strong_buy
114,MSFT,Fortress,459.860,32.68,Technology,9.21,26.37,48.87,43.52,1.401,52.84,3418.22,20.65,22.16,6.81,1.27586,strong_buy
65,YUMC,Fortress,47.550,19.73,Consumer Cyclical,3.83,13.58,12.60,8.82,1.320,100.00,16.85,10.24,14.37,28.74,1.33333,strong_buy
91,META,Fortress,620.250,27.47,Communication Services,12.30,29.44,40.08,35.32,1.978,99.83,1563.36,15.96,19.16,16.70,1.35821,strong_buy
123,IMAX,Fortress,37.430,52.72,Communication Services,3.17,7.38,27.25,7.82,3.942,5.66,2.01,18.76,19.16,2.09,1.36364,strong_buy


In [39]:
# ==========================================
# 13. THE "DEEP VALUE" INTERSECTION (Buffett + Burry)
# ==========================================

# 1. Ensure we have the Buffett Data
if 'Buffett_Value_DF' not in locals():
    print("Buffett Data not found. Running scan now...")
    if 'get_buffett_value_picks' in globals() and 'final_results' in locals():
        Buffett_Value_DF = get_buffett_value_picks(final_results)
    else:
        print("Missing 'final_results' or 'get_buffett_value_picks' function.")
        Buffett_Value_DF = pd.DataFrame()

# 2. Ensure we have the Burry Data
if 'Fortress_Burry_EV_EBITDA' not in locals():
    print("Please run the Burry EV/EBITDA filter cell first.")
    Fortress_Burry_EV_EBITDA = pd.DataFrame()

# 3. THE MERGE (Finding the Overlap)
if not Buffett_Value_DF.empty and not Fortress_Burry_EV_EBITDA.empty:
    
    # Merge on Ticker to find stocks that appear in BOTH lists
    # We use an 'inner' join, which means "keep only if in both"
    Deep_Value_Intersection = pd.merge(
        Buffett_Value_DF[['Ticker', 'P/B Ratio', 'ROE %', 'Debt/Eq %']], 
        Fortress_Burry_EV_EBITDA[['Ticker', 'Price', 'Sector', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%', 'Tier']],
        on='Ticker', 
        how='inner'
    )
    
    if not Deep_Value_Intersection.empty:
        print("\n" + "="*60)
        print(f"DEEP VALUE GEMS FOUND: {len(Deep_Value_Intersection)}")
        print("="*60)
        print("Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)")
        
        # Sort by the "Discount" (how cheap they are vs sector)
        Deep_Value_Intersection = Deep_Value_Intersection.sort_values(by='Discount_%', ascending=False)
        
        cols = ['Ticker', 'Price', 'Tier', 'P/B Ratio', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%', 'Sector']
        display(Deep_Value_Intersection[cols])
        
    else:
        print("\n No stocks passed BOTH filters.")
        print("This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time.")
        print(f"Buffett Count: {len(Buffett_Value_DF)} | Burry Count: {len(Fortress_Burry_EV_EBITDA)}")

else:
    print("Cannot combine. One of the filters returned 0 results.")


DEEP VALUE GEMS FOUND: 2
Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)


,Ticker,Price,Tier,P/B Ratio,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Sector
0,TGNA,18.85,Fortress,0.97,7.73,19.16,59.66,Communication Services
1,TMHC,63.27,Fortress,1.00,6.11,14.37,57.48,Consumer Cyclical


In [ ]:
# =============================================================================
# CELL 14: THE "HIGH CONVICTION" INTERSECTION (BURRY + INSIDERS)
# =============================================================================
# This finds stocks that are BOTH:
# 1. Undervalued vs their Sector (Burry EV/EBITDA)
# 2. Being accumulated by Insiders (Insider Confidence)
#
# This is a powerful "Anti-Value Trap" filter. Insiders rarely buy 
# sinking ships, even if they look cheap on paper.

# --- 1. DETECT BEST AVAILABLE DATAFRAMES ---

# Try to find the Insider list (preferring the one with Analyst ratings if available)
if 'Fortress_insiders_Analyst_buy' in locals() and not Fortress_insiders_Analyst_buy.empty:
    df_insider_source = Fortress_insiders_Analyst_buy
    print("Using Analyst-Vetted Insider List.")
elif 'Fortress_insiders' in locals() and not Fortress_insiders.empty:
    df_insider_source = Fortress_insiders
    print("Using Raw Insider List (No Analyst Check).")
else:
    df_insider_source = pd.DataFrame()
    print("Warning: No Insider data found.")

# Try to find the Burry list (preferring the one with Analyst ratings if available)
if 'Fortress_Burry_Analyst_buy' in locals() and not Fortress_Burry_Analyst_buy.empty:
    df_burry_source = Fortress_Burry_Analyst_buy
    print("Using Analyst-Vetted Burry List.")
elif 'Fortress_Burry_EV_EBITDA' in locals() and not Fortress_Burry_EV_EBITDA.empty:
    df_burry_source = Fortress_Burry_EV_EBITDA
    print("Using Raw Burry List (No Analyst Check).")
else:
    df_burry_source = pd.DataFrame()
    print("Warning: No Burry EV/EBITDA data found.")

# --- 2. PERFORM THE INTERSECTION ---

if not df_insider_source.empty and not df_burry_source.empty:
    
    # We merge on 'Ticker'. 'inner' means keep only stocks found in BOTH lists.
    # We use suffixes=('', '_drop') to handle duplicate columns (like Price)
    high_conviction_df = pd.merge(
        df_burry_source[['Ticker', 'Price', 'Sector', 'Tier', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%']],
        df_insider_source[['Ticker', 'Insider_Buys_Count', 'Net_Shares_Bought']],
        on='Ticker',
        how='inner'
    )
    
    if not high_conviction_df.empty:
        # Sort by Discount % (Deepest value first)
        high_conviction_df = high_conviction_df.sort_values(by='Discount_%', ascending=False)
        
        print("\n" + "="*60)
        print(f"🚀 HIGH CONVICTION WINNERS FOUND: {len(high_conviction_df)}")
        print("="*60)
        print("Criteria: Cheap vs Sector (Burry) + Insider Buying")
        
        # Reorder columns for readability
        cols = [
            'Ticker', 'Price', 'Sector', 'Tier', 
            'Discount_%', 'EV/EBITDA', 
            'Insider_Buys_Count', 'Net_Shares_Bought'
        ]
        
        # Display nicely
        display(high_conviction_df[cols])
        
        # Optional: Save to CSV
        save_path = os.path.join(DATA_FOLDER, "High_Conviction_Picks_USA.csv")
        high_conviction_df.to_csv(save_path, index=False)
        print(f"\nSaved to: {save_path}")
        
    else:
        print("\nNo overlap found. No stock is both 'Cheap vs Sector' AND 'Insider Buy' target.")
        print(f"Burry Count: {len(df_burry_source)} | Insider Count: {len(df_insider_source)}")
else:
    print("\nCannot run intersection. Missing one or both source lists.")

Using Analyst-Vetted Insider List.
Using Analyst-Vetted Burry List.

🚀 HIGH CONVICTION WINNERS FOUND: 8
Criteria: Cheap vs Sector (Burry) + Insider Buying


,Ticker,Price,Sector,Tier,Discount_%,EV/EBITDA,Insider_Buys_Count,Net_Shares_Bought
7,FSM,10.42,Basic Materials,Fortress,69.76,4.83,10,2116207.0
6,CGAU,16.13,Basic Materials,Fortress,60.61,6.29,87,4297682.0
0,OPCH,36.03,Healthcare,Fortress,20.19,16.64,8,18954.0
3,MU,362.75,Technology,Fortress,16.70,18.46,3,167433.0
5,RAL,53.53,Technology,Fortress,14.67,18.91,1,2000.0
2,CRH,122.97,Basic Materials,Fortress,13.96,13.74,3,413316.0
1,GIL,63.22,Consumer Cyclical,Fortress,9.39,13.02,105,1426104.0
4,SHLS,9.27,Technology,Fortress,7.54,20.49,3,8335.0



Saved to: YfinanceDataDump\High_Conviction_Picks_ca.csv


In [68]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['SHLS','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD','SVM','GIL'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 12 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
0,ADMA,17.31,-1.09,17.92,-32.57,1.00,30.00,0.93,3.17
9,SHLS,9.27,-1.07,6.10,-18.40,1.90,10.75,1.60,5.34
11,UBER,84.85,0.56,85.46,-16.81,1.47,111.17,1.11,2.40
6,MIR,26.93,-0.26,20.35,-11.05,1.12,30.50,0.75,3.51
4,FLEX,64.83,-2.64,49.43,-10.23,1.42,75.00,0.47,2.93
5,GIL,63.22,-0.92,52.67,-3.38,1.14,73.80,0.92,2.08
3,DD,42.86,-1.22,32.51,-2.97,1.48,48.19,0.88,2.24
7,ONB,23.26,-0.13,21.50,-2.58,1.85,26.23,1.98,2.11
8,SEI,56.87,2.65,33.68,-1.91,1.17,66.27,1.41,5.91
2,BANC,20.80,-0.34,15.63,-1.38,1.27,22.59,1.02,2.12


In [69]:
tickers_gemini = ['GIL'] 
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = r"C:\Users\jdcc3\OneDrive - McMaster University\Gemini API Key\gemini_key.txt"

def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"Error reading key file: {e}")
        return None

# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("API Key loaded securely.")
else:
    print("CRITICAL: API Key not loaded. The script will fail.")

# ==========================================
# SENTIMENT ANALYSIS FUNCTION USING GROUNDING SEARCH GEMINI 3
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):
    
    if not os.environ.get("GEMINI_API_KEY"):
        print("Stop: No API Key found.")
        return

    print(f"\nGemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model='gemini-3-flash-preview', # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config
        )
        display(Markdown(response.text))
        
    except Exception as e:
        print(f"Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

API Key loaded securely.

Gemini 3 is thinking (High Reasoning Mode)... analyzing $['GIL']...


## Gemini 3 Sentiment Report: Gildan Activewear (GIL)
**Reasoning Depth:** High  
**Sentiment Score:** 8.2/10  
**Verdict:** Buy / Opportunity

### 1. The Bull Thesis (Why it goes up)
*   **Transformative Acquisition**: The completion of the $2.2 billion HanesBrands acquisition (December 2025) is a "game-changer," effectively doubling Gildan's scale and revenue to a pro-forma ~$7 billion.
*   **Synergy Capture**: Management has guided for $200 million in annual run-rate cost synergies within three years. This is expected to drive a 20% compounded annual growth rate (CAGR) in EPS through 2028.
*   **Aggressive Price Targets**: Top-tier analysts (UBS) recently hiked price targets to $110, implying significant upside (~70%+) from current levels, citing the Hanes deal as a catalyst for a valuation re-rating.
*   **Return to Stability**: Following the high-profile proxy battle and the reinstatement of CEO Glenn Chamandy, the "governance discount" has dissipated, replaced by a clear focus on the "Gildan Sustainable Growth" (GSG) strategy.

### 2. The Bear Thesis (Why it goes down)
*   **Integration and Debt Risk**: Pro-forma leverage has spiked to approximately 3.2x EBITDA. Any hiccups in integrating HanesBrands’ complex innerwear business or failure to meet synergy targets could weigh on the stock.
*   **Regulatory & Antitrust Drag**: While the deal has closed, residual antitrust scrutiny regarding dominance in the "basic apparel" and "printwear" channels remains a headline risk, potentially necessitating further divestitures.
*   **Market Complacency**: Technical analysts warn of "complacent dip buying" as the stock hits fresh highs, with RSI levels suggesting it may be over-extended in the short term ($68-$70 resistance).
*   **Macro Headwinds**: Exposure to cotton price volatility and potential tariff shifts (75% of Hanes' sales were previously impacted by various trade policies) remains a permanent background risk for the apparel sector.

### 3. Deep Dive Analysis

#### **News Analysis**
Headline sentiment is overwhelmingly **euphoric** regarding the HanesBrands merger. Viral coverage focuses on Gildan becoming a "global apparel titan." However, a secondary stream of more cautious "fear-mongering" exists in legal and governance circles, questioning if HanesBrands was undervalued or if the board's independence was compromised during the auction. In the last 30 days, the narrative has shifted from "Will it close?" to "How fast can they deleverage?"

#### **Smart Money**
Institutional ownership is exceptionally high at **~82-86%**, signaling that "Smart Money" has already cornered the float. Recent filings show a mix of consolidation; while some large funds like FMR LLC trimmed positions in late 2024, CIBC and others have been aggressive buyers. Insider ownership remains low (~2%), but the return of CEO Glenn Chamandy is viewed by the market as a "synthetic" insider buy due to his deep alignment with the company's vertically integrated model.

#### **Financial Statement Analysis**
*   **Historic Performance (2022-2024)**: Revenue has been stable but sluggish, moving from $3.24B (2022) to $3.27B (2024). Profitability took a hit in 2024 (earnings down ~24%) due to integration costs and the proxy fight.
*   **Forward Outlook (2025-2026)**: 2025 is the transition year. Revenue is projected to jump nearly 100% following the Hanes integration. 
*   **Profitability**: Current net margins of 14.1% are expected to expand as manufacturing efficiencies from Hanes’ plants are optimized under Gildan's low-cost "hub" model.

#### **The "Whisper" Number**
The official analyst consensus for the upcoming Q4 earnings (Feb 18, 2026) is an EPS of **$0.93**. However, the "whisper" among traders on forums like StockTwits and Reddit leans slightly more bullish at **$0.96 - $0.98**, driven by expectations of early synergy wins and strong holiday printwear demand.

### 4. Conclusion
The current price of GIL is an **opportunity**, not a trap. While technical indicators suggest a short-term "cooling off" period or a minor pullback to the $62-$63 support zone, the fundamental transformation is too large to ignore. Gildan has transitioned from a steady-state commodity apparel maker into a dominant global platform with a clear path to $6+ EPS by 2028. **The market is currently pricing in the risk of the deal but hasn't yet fully priced in the scale of the rewards.**

In [ ]:
tickers_gemini = ['SHLS'] 
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# Senior Analyst with INTERACTIVE FOLLOW-UPS
# ==========================================

KEY_FILE_PATH = r"C:\Users\jdcc3\OneDrive - McMaster University\Gemini API Key\gemini_key.txt"

def load_api_key(filepath):
    try:
        with open(filepath, "r") as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error reading key file: {e}")
        return None

api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("API Key loaded securely.")
else:
    print("CRITICAL: API Key not loaded.")

# ==========================================
# INTERACTIVE ANALYSIS FUNCTION
# ==========================================
def analyze_and_chat_gemini_3(tickers_gemini, company_name=None):
    
    if not os.environ.get("GEMINI_API_KEY"):
        print("Stop: No API Key found.")
        return

    print(f"\nGemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # 1. Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    # 2. Configure the Chat with Tools (Search) and Reasoning
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    # 3. Create the Chat Session (This holds the history)
    chat = client.chats.create(
        model='gemini-3-pro-preview', # Recommended for reasoning + speed
        config=config
    )

    # 4. Define the Initial Prompt
    initial_prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini}.
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric?
    2. **Analyst Shifts**: Are price targets moving UP or DOWN?
    3. **Institutional Flows**: Hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## Gemini Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis
    * ...
    
    ### 2. The Bear Thesis
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance last 3y + outlook)
    
    ### 4. Conclusion
    """

    try:
        # 5. Send Initial Request
        response = chat.send_message(initial_prompt)
        display(Markdown(response.text))
        
        # ==========================================
        # NEW: INTERACTIVE LOOP
        # ==========================================
        print("\n" + "="*50)
        print(f" INTERACTIVE SESSION: Ask follow-ups about {tickers_gemini}")
        print(" Type 'exit' or 'quit' to stop.")
        print("="*50 + "\n")

        while True:
            # Get user input
            user_question = input("\nYour Question: ")
            
            # Check for exit condition
            if user_question.lower() in ['exit', 'quit', 'stop']:
                print("Ending session.")
                break
            
            # Send follow-up to the SAME chat session
            print("Analyzing...")
            follow_up_response = chat.send_message(user_question)
            display(Markdown(follow_up_response.text))

    except Exception as e:
        print(f"Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
if os.environ.get("GEMINI_API_KEY"):
    analyze_and_chat_gemini_3(tickers_gemini, tickers_gemini)

API Key loaded securely.

Gemini 3 is thinking (High Reasoning Mode)... analyzing $['SHLS']...


## Gemini Sentiment Report: ['SHLS']

**Ticker:** SHLS (Shoals Technologies Group, Inc.)  
**Date:** January 19, 2026  
**Current Price:** ~$9.63  
**Reasoning Depth:** High  
**Sentiment Score:** 7/10 (Cautiously Optimistic)  
**Verdict:** **Speculative Buy**

---

### 1. The Bull Thesis
*   **The "Insider Floor":** The most potent signal is the aggressive open-market purchasing by top brass. CEO Brandon Moss and CFO Dominic Bardos executed significant buys in late 2024 and early 2025 (e.g., CFO bought ~$98k worth in March 2025). When executives buy a "falling knife" or a stagnant stock, it signals they believe the market's pessimism is detached from fundamental reality.
*   **Patent War 2.0 (Offense vs. Defense):** While the initial ITC ruling regarding the '153 patent was a setback, Shoals has successfully pivoted to offense. By filing a new ITC complaint based on *newly issued* patents ('375 and '376) and appealing the previous decision to the Federal Circuit, they have extended the timeline and uncertainty for competitors like Voltage. The market is beginning to price in a "settlement" or "victory by attrition" scenario rather than total defeat.
*   **Stabilization & Beat:** Q3 2025 revenue ($135.8M) beat expectations, and the company raised its full-year guidance. After a revenue contraction in 2024 (-18%), the return to growth and margin stability suggests the EBOS (Electrical Balance of Systems) inventory destocking cycle is finally over.

### 2. The Bear Thesis
*   **Litigation Overhang:** The Voltage legal battle is not just a headline; it's an existential risk to Shoals' "moat." If Voltage’s alternative designs (LYNX) are definitively ruled non-infringing and the new patents fail to stick, Shoals faces rapid commoditization. The "premium" multiple they command relies on their IP protection.
*   **Valuation Friction:** Even at ~$9.63, SHLS trades at a P/E of ~48x. This is a growth multiple in a sector (Solar/Industrials) that has faced interest rate headwinds. If growth doesn't re-accelerate to >20% annually, this multiple is vulnerable to compression.
*   **Analyst Fatigue:** While ratings are generally "Buy," the lack of *new* massive price target hikes suggests Wall Street is in "wait and see" mode regarding the litigation outcomes in late 2026.

### 3. Deep Dive Analysis

#### **News Virality: "The Empire Strikes Back"**
The news cycle has shifted from fear (initial patent loss) to a "battleground" narrative. Headlines regarding the **Federal Circuit Appeal** and the **New ITC Investigation** (instituted Feb 2025) have drowned out the previous bearish noise. The tone is no longer "Shoals is losing market share" but rather "Shoals is fiercely defending its territory." This shift from victim to aggressor is key for sentiment.

#### **Analyst Shifts: Holding the Line**
*   **Consensus:** Moderate Buy / Hold.
*   **Price Target Trend:** Targets have stabilized in the **$11.00 – $15.00** range.
    *   *Recent Moves:* UBS ($12), Guggenheim ($12), and Goldman Sachs ($11) have maintained or slightly lifted targets, offering ~20-50% upside from current levels.
    *   *Note:* Morgan Stanley remained cautious with a $9.50 target in late 2025, essentially calling the stock "dead money" until the legal clouds clear.
*   **Takeaway:** Analysts are not capitulating. They are keeping targets above current price, acting as a soft psychological floor.

#### **Institutional & Insider Flows: The "Smart Money" Divergence**
*   **Insiders (Bullish):** As noted, the C-Suite buying is the strongest pillar of the Bull Thesis. Insiders rarely buy for a 10% bounce; they buy for multi-bagger returns or to defend the stock from collapsing.
*   **Institutions (Mixed to Bullish):** Institutional inflows over the last 12 months ($470M) have more than doubled outflows ($200M). Despite the volatility, funds are seemingly using the dip to accumulate, betting on the long-term secular trend of solar installation growth.

#### **The "Whisper" Number: Retail vs. Reality**
*   **Forum Sentiment (Reddit/StockTwits):** Retail sentiment is recovering. The "bagholder" despair of 2024 has transitioned into "accumulation" talk.
*   **The Whisper:** Traders are eyeing a break above **$10.50**. The chatter suggests that if SHLS wins *any* leg of the current patent appeals, the stock could swiftly re-rate to **$15+**. Conversely, the downside is viewed as capped near $7.50 (where insiders stepped in previously).

#### **Financial Health Check**
*   **Revenue:** 2024 was a wash (-18% decline), but 2025 showed signs of a pivot (Q3 beat).
*   **Margins:** Gross margins remain healthy (~35%), indicating they haven't had to slash prices drastically to compete with Voltage *yet*.
*   **Cash:** Balance sheet is adequate to fund the legal war chest without immediate dilution risk.

### 4. Conclusion

**Shoals Technologies (SHLS)** is currently a "battleground stock" undergoing a sentiment reversal. The market effectively priced in the "worst-case scenario" (total loss of patent exclusivity) when the stock hit its lows. Now, with the C-Suite putting their own money on the table and a renewed legal offensive, the risk/reward profile is skewed to the upside.

**Verdict:** **Speculative Buy**.
*   *Entry:* Under $9.75 is attractive.
*   *Stop Loss:* A close below $8.00 (invalidates the "Insider Floor" thesis).
*   *Catalyst:* Updates on the Federal Circuit appeal or preliminary wins in the *new* ITC investigation (expected mid-to-late 2026).


 INTERACTIVE SESSION: Ask follow-ups about ['SHLS']
 Type 'exit' or 'quit' to stop.

Analyzing...


The recent wave of "high-conviction" insider buying for Shoals Technologies (SHLS) began in **November 2024**, signaling the executives' belief that the stock was undervalued.

The specific timeline of these purchases confirms the "Insider Floor" thesis:

*   **The Catalyst Buy (Nov 21, 2024):** CEO Brandon Moss initiated the trend by purchasing **22,300 shares** at an average price of **$4.55**.
*   **The Follow-Up (Nov 22, 2024):** The very next day, CFO Dominic Bardos bought **15,000 shares** at **$4.71**.
*   **The "Double Down" (March 12, 2025):** When the stock price dipped significantly in early 2025, CFO Dominic Bardos returned to the market to buy a larger tranche—**35,000 shares** at just **$2.80** (a ~$98,000 investment).

**Why this matters now:**
Given the stock is trading around **$9.63** (as of Jan 2026), these insiders have effectively timed the bottom. Their purchases at ~$2.80–$4.70 are currently up **~100% to 240%**, providing a strong psychological signal to the market that leadership can correctly value their own company against external pessimism.

Analyzing...


...

Analyzing...


**Research session closed.**

System shutting down. Good luck with your trades.

Analyzing...


**Session Terminated.**

Analyst reporting offline. Good luck with your trades.

Ending session.
